# python连接Oracle数据库

In [1]:
import pandas as pd
import numpy as np
import cx_Oracle
import os
connection=cx_Oracle.connect('sys','linzch2020','202.38.228.21:1521/ORCL',cx_Oracle.SYSDBA)

# 统计物流园区的吸引度

In [2]:
# 遍历六月份的gps数据，根据起始点经纬度，计算每个物流园区的到发量（由距离公式，如果到圆心距离小于3km，则属于从该点出发或到达该点）
s1='select startpx,startpy,endpx,endpy from GPS_OUTPUT_RAW_202106_T1 group by startpx,startpy,endpx,endpy'
car_SE=pd.read_sql(s1,connection)

In [3]:
car_SE=car_SE[['STARTPX','STARTPY','ENDPX','ENDPY']].values
car_SE

array([[113.202451,  23.341009, 113.440784,  22.55662 ],
       [113.114595,  23.039306, 113.040098,  22.959141],
       [113.528264,  23.1292  , 113.522968,  23.055634],
       ...,
       [113.317904,  23.433116, 113.318424,  23.433166],
       [113.41789 ,  22.895571, 113.506613,  22.86398 ],
       [113.240886,  23.280304, 113.239764,  23.285535]])

In [4]:
hub_info=pd.read_excel('广东省物流园区信息(坐标更新版).xlsx')
hub_info=hub_info[['经度','维度']]
hub_info=hub_info.values
type(hub_info)

numpy.ndarray

In [5]:
o=np.zeros(380)
d=np.zeros(380)
type(o)

numpy.ndarray

In [6]:
import pydistance #导入该py，wgs84坐标系下经纬度计算距离。
for i in range(car_SE.shape[0]):
    for j in range(hub_info.shape[0]):
        car_slng,car_slat=car_SE[i,0],car_SE[i,1]#获取每次出行记录的起始点经纬度
        car_elng,car_elat=car_SE[i,2],car_SE[i,3] #终点经纬度
        hub_lng,hub_lat=hub_info[j,0],hub_info[j,1] #物流园区经纬度
        s_distance=pydistance.get_distance_wgs84(car_slng,car_slat,hub_lng,hub_lat) #计算距离
        e_distance=pydistance.get_distance_wgs84(car_elng,car_elat,hub_lng,hub_lat)
        if(s_distance<=3000): #如果小于3000m，则计数
            o[j]+=1
        if(e_distance<=3000):
            d[j]+=1
        #print(i,j)

# 将计算结果从ndarry转化为dataframe

In [24]:
def ndarry_to_df(nd:np.ndarray,s:str):
    df=pd.DataFrame(nd)
    df=df/30
    df.columns=[s]
    df[s]=df[s].astype(np.int64)
    return df

In [30]:
hub_info=pd.read_excel('广东省物流园区信息(坐标更新版).xlsx')
df_o=ndarry_to_df(o,'货车出发频次')
hub_info['货车出发频次']=df_o['货车出发频次']
df_d=ndarry_to_df(d,'货车到达频次')
hub_info['货车到达频次']=df_d['货车到达频次']
writer=pd.ExcelWriter('广东省物流园区货车频次.xlsx')
hub_info.to_excel(writer)
writer.save()

In [34]:
o.max(),o.sum()

(16946.0, 937324.0)

In [38]:
o.sum(axis=0)

937324.0